In [ ]:
! pip install networkx[default]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import networkx as nx

Inizio

In [ ]:
import sqlite3

connection = sqlite3.connect('AAD.db')
connection.row_factory = sqlite3.Row
posts = connection.execute('SELECT * FROM Attacks').fetchall()
connection.close()

In [ ]:
def getOnlyId (s):
  if "Step1" in s:
    b = s.replace("_Step1","")
  else:
    a = s[0:s.rfind('_')]
    b = a[0:a.rfind('_')]
  return b

In [ ]:
listgraph = []

for post in posts:
  if "SSA" in post['ID']:
    listgraph.append( nx.DiGraph([("r", post['ID'])]))  

  elif ("MSA" and "Step1") in post['ID']:

    group = []
    listofTuple = []
    for post2 in posts:
      if getOnlyId (post['ID']) == getOnlyId (post2['ID']):
        group.append(post2['ID'])
    
    print (group)
    step1 = next((s for s in group if "Step1" in s), None)
    listofTuple.append( ("r", step1) )

    nextStep = [s for s in group if "_1" in s]
    for x in nextStep:
      listofTuple.append( (step1, x) )

    i=2
    while i<=10:
      precStep = nextStep
      nextStep = [s for s in group if ("_"+str(i)) in s]
      for y in precStep:
        for x in nextStep:
          listofTuple.append( (y, x) ) 
      i=i+1


    print ("Tuple: ", listofTuple)
    listgraph.append ( nx.DiGraph( listofTuple ) )

      

    print ("*****")


In [ ]:
len(listgraph)

361

In [ ]:
R = nx.compose_all(listgraph)

In [ ]:
roots = (v for v, d in R.in_degree() if d == 0)
leaves = (v for v, d in R.out_degree() if d == 0)
all_paths = []
for root in roots:
    for leaf in leaves:
        paths = nx.all_simple_paths(R, root, leaf)
        all_paths.extend(paths)
all_paths

In [ ]:
print (len(all_paths))

372


In [ ]:
import csv

with open("out.csv", "w", encoding='utf-8') as f:
    wr = csv.writer(f)
    wr.writerows(all_paths)